In [1]:
import ipyvuetify as v
import ipywidgets
from scripts import zonal_computation as zc

In [2]:
#input 
country = None

In [3]:
#map widget
import ee
import geemap

ee.Initialize()

m = geemap.Map()

In [4]:
list_country = {
    'brésil': 'BR', 
    'colombie': 'CO', 
    'peru': 'PE', 
    'cameroun': 'CM', 
    'congo': 'CF', 
    'rdc': 'CG', 
    'gabon': 'GB', 
    'Ouganda': 'UG', 
    'ghana': 'GH', 
    "côte d'Ivoire": 'IV', 
    'indonésie': 'ID',
    'malaisie': 'MY' 
}

In [5]:
aoi_country_selection = v.Select(items=[*list_country], label='Country/Province', v_model=None, class_="d-inline")

def on_country_change(widget, event, data):
    global country_selection

    country = list_country[widget.v_model]

aoi_country_selection.on_event('change', on_country_change )

In [6]:
#validate the selected data 
aoi_select_btn = v.Btn(
    color='primary', 
    children=[
        v.Icon(left=True, children=['mdi-map-marker-check']),
        'Select these inputs'
    ]
)

def aoi_on_click(widget, event, data):
    
    #io variables 
    global country
    
    #do something 
    #utils.toggleLoading(aoi_select_btn)
    figs = zc.run_zonal_computation(country, m)
    #update the figs in the results tile
    process_hist.children = figs
    
    #utils.toggleLoading(aoi_select_btn)
    
    

aoi_select_btn.on_event('click', aoi_on_click)

In [7]:
#create the display 
inputs = v.Layout(
    _metadata={'mount-id': 'data-input'},
    class_="pa-5",
    row=True,
    align_center=True, 
    children=[
        v.Flex(xs12=True, children=[aoi_country_selection]),
        aoi_select_btn
    ]
)
#inputs

AOI_content_main =  v.Layout(
    dark=True,
    _metadata={'mount_id': 'step-1-widget'},
    xs12=True,
    row=True,
    class_="ma-5 d-inline",
    children=[
        v.Card(
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Selectionner le pays']),
                v.Layout(
                    row=True,
                    xs12=True,
                    children=[
                        v.Flex(xs12=True, md6=True, children=[inputs]),
                        v.Flex(class_="pa-5", xs12=True, md6=True, children=[m])
                    ]
                )    
            ]
        )
    ]
)

In [8]:
footer = v.Footer(
    color='#2e7d32',
    class_="white--text",
    app = True,
    children = ['SEPAL \u00A9 2020']
)

In [9]:
appBar = v.AppBar(
    color="#2e7d32",
    class_="white--text",
    dense=True,
    app = True,
    children = [v.ToolbarTitle(children=['Zonal analysis'])]
)

In [10]:
download_csv = v.Btn(
    class_='ma-2',
    xs5=True,
    color='primary',
    href='#',
    children=[
        v.Icon(left=True, children=['mdi-download']),
        'download .csv'
    ]
)

process_hist = v.Flex(xs12=True, lg6=True, class_='pa-0', children=['no result yet'])

results = v.Layout(
    _metadata={'mount_id': 'widget_result'},
    Row=True,
    xs12=True,
    align_center=True,
    class_="ma-5 d-inline",
    children=[
        v.Card( 
            class_="pa-5",
            raised=True,
            xs12=True,
            children=[
                v.Html(xs12=True, tag='h2', children=['Results']),
                v.Layout(Row=True, xs12=True, children=[
                    download_csv
                ]),
                process_hist  
            ]
        )
    ]
)

In [11]:
content = v.Content(
    children=[
        v.Container(
            fluid=True,
            children = [
                AOI_content_main, 
                results
            ]
        )
    ]
)

In [12]:
app = v.App(v_model=None)
app.children = [appBar, content, footer]
app

App(children=[AppBar(app=True, children=[ToolbarTitle(children=['Zonal analysis'])], class_='white--text', col…

In [13]:
m

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:

country = 'CG'
zc.run_zonal_computation(country, m)


Computing statistics ...
Generating URL ...
